<a href="https://colab.research.google.com/github/haonanzhang314/Machine-Learning/blob/master/%E6%89%8B%E5%86%99%E6%95%B0%E5%AD%97%E8%AF%86%E5%88%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:

"""手写数字的识别流程
准备数据集包含训练集和测试集
将数据集的图片做尺寸归一化，保持相同大小
1、加载必要的库：nn网络库、优化器
2、定义超参数//例如定义循环的次数。
#batch_size批处理。每次处理的数据数量,意思为将海量的数据分批输入电脑中减少压力，一般设置为64,128，根据设备的性能调节
#如果有GPU则用GPU训练，否则用cpu训练
#EPOCHS:训练数据集的轮次，意思是你可以选择将6万张照片训练一次、10次、100次。。。。
3、构建transforms，主要是对图像做变换
#构建pipeline也就transfoms,对图像进行处理
#可以对图片转换成tensor，旋转图片，以及正则化，明亮度，等等进行处理
#将图片转换成tensor格式
# nomalize正则化。模型出现过拟合现象时，降低模型复杂度
# 过拟合的含义：例如你训练出的模型只认识你自己写的字迹，你朋友写的字迹就不认识了。只认识见过的一模一样的，稍微改变一点就不认识了。
transforms.Normalize((0.1307,), (0.3081,)) #正则化，如果不确定取多少值，那就选择这个官网给的值
4、下载、加载数据集MNIST
#dataloader对数据进行处理
#下载数据集
#train训练集train是true，test集train就要是false假了，transform=pipline是将数据transfom对数据处理
#加载数据
#shuffle=True是将图片打乱，是训练图片无顺序，有助于模型精度提高
5、构建网络模型------------重要
#构建名叫Digit网络模型继承nn.Module
#卷积层第1层 1: 灰度图片通道 10：输出通道 5：kernel卷积核
#卷积层第2层 10：输入通道 20输出通道 3：kernel卷积核
#全连接层1 20*10*10：输入通道 ，500输出通道
#全连接层2 500:输入通道 10输出通道
# 前向传播
# batch_size*1*28*28为1灰度28是像素
# batch*1*28*28 ，输出：batch*10*24*24   24是28-5+1=24
# 激活函数的作用就是，在所有的隐藏层之间添加一个激活函数，这样的输出就是一个非线性函数，神经网络的表达能力就更加强大了 保持shpae不变，输出：batch*10*24*24
# 池化层，将特征更加明显，不明显的忽略。类似于降噪。 输入：batch*10*24*24 输出：batch*10*12*12
# 卷积层二层 输入：batch*10*12*12 输出层：batch*20*10*10 （12-3+1=10）
# 拉伸，将矩阵形式的图片拉伸成一列 -1自动计算维度，20*10*10=2000
# 输入：batch*2000 输出：batch*500
# 输入：batch*500 输出：batch*10
# 计算分类后，每个数字的概率值
#返回概率值
6、定义优化器
#优化器的作用是更新模型的参数，使得最终的结果达到最优值
7、定义训练方法
# 模型训练
# 把训练部署到DEVICE上
# 梯度初始化为0
# 训练后的结果
# 计算损失
# 反向传播
# 参数优化
# 打印
8、定义测试方法
# 模型验证
# 统计正确率
# 测试损失
#此处不计算梯度，也不反向传播
#部署到device上
# 测试数据
# 计算测试损失
# 找到概率值最大的下标
# 累计正确的值
9、调用方法 ，开始训练模型."""
"""专业名词解释
    1、参数与超参数
    参数：模型f（x,$）中的$称为模型的参数，可以通过优化算法进行学习。
    超参数：用来定义模型结构或优化策略。
    2、batch_size批处理。每次处理的数据数量
    3、epoch轮次。把一个数据集，循环运行几轮。
    4、transforms变换。主要是将图片转换为tensor，旋转图片，以及正则化
    5、nomalize正则化。模型出现过拟合现象时，降低模型复杂度。
    6、卷积层。由卷积核构建，卷积核简称为卷积，也称为滤波器。卷积的大小可以在实际需要的时候自定义其长和宽（1*1,3*3，5*5）
    7、池化层。对图片进行压缩（降采样）的一种方法，如max pooling，average pooling等
    8、激活层。激活函数的作用就是，在所有隐藏层之间添加一个激活函数，这样的输出就是一个非线性函数了，因而神经网络的表达能力就更加强大了
    9、损失函数。在深度学习中，损失函数反应了模型最后预测效果和实际真值之间的差距，可以用来分析训练过程的好坏、模型是否收敛等、
    例如均方损失、交叉熵损失"""

#加载必要的库
import torch
#nn网络库
import torch.nn as nn
import torch.nn.functional as F
#优化器
import torch.optim as optim
from torchvision import datasets, transforms

#定义超参数
#batch_size批处理。每次处理的数据数量,
#意思为将海量的数据分批输入电脑中减少压力，一般设置为64,128，根据设备的性能调节
BATCH_SIZE = 64
#如果有GPU则用GPU训练，否则用cpu训练
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#训练数据集的轮次，意思是你可以选择将6万张照片训练一次、10次、100次。。。。
EPOCHS = 20

#构建pipeline也就transfoms,对图像进行处理
#可以对图片转换成tensor，旋转图片，以及正则化，明亮度，等等进行处理
pipeline = transforms.Compose([
    transforms.ToTensor(),#将图片转换成tensor格式
    #nomalize正则化。模型出现过拟合现象时，降低模型复杂度
    #过拟合的含义：例如你训练出的模型只认识你自己写的字迹，你朋友写的字迹就不认识了。只认识见过的一模一样的，稍微改变一点就不认识了。
    transforms.Normalize((0.1307,), (0.3081,)) #正则化，如果不确定取多少值，那就选择这个官网给的值
])

#下载、加载数据·
#dataloader对数据进行处理
from torch.utils.data import DataLoader
#下载数据集
#train训练集train是true，test集train就要是false假了，transform=pipline是将数据transfom对数据处理
train_set = datasets.MNIST("data", train=True, download=True, transform=pipeline)
test_set =datasets.MNIST("data", train=False, download=True, transform=pipeline)
#加载数据
#shuffle=True是将图片打乱，是训练图片无顺序，有助于模型精度提高
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True)

#构建网络模型
class Digit(nn.Module): #构建名叫Digit网络模型继承nn.Module
    def __init__(self): #构造方法
        super().__init__() #调用方法
        self.conv1 = nn.Conv2d(1, 10, 5)#卷积层第1层 1: 灰度图片通道 10：输出通道 5：kernel卷积核
        self.conv2 = nn.Conv2d(10, 20, 3)#卷积层第2层 10：输入通道 20输出通道 3：kernel卷积核
        self.fc1 = nn.Linear(20*10*10, 500)#全连接层1 20*10*10：输入通道 ，500输出通道
        self.fc2 = nn.Linear(500, 10)#全连接层2 500:输入通道 10输出通道

        #前向传播
    def forward(self, x):
        input_size = x.size(0) #batch_size*1*28*28为1灰度28是像素
        x = self.conv1(x) #batch*1*28*28 ，输出：batch*10*24*24   24是28-5+1=24
        x = F.relu(x)#激活函数的作用就是，在所有的隐藏层之间添加一个激活函数，这样的输出就是一个非线性函数，神经网络的表达能力就更加强大了 保持shpae不变，输出：batch*10*24*24
        x = F.max_pool2d(x, 2, 2) #池化层，将特征更加明显，不明显的忽略。类似于降噪。 输入：batch*10*24*24 输出：batch*10*12*12


        x = self.conv2(x)#卷积层二层 输入：batch*10*12*12 输出层：batch*20*10*10 （12-3+1=10）
        x = F.relu(x)

        x = x.view(input_size, -1)#拉伸，将矩阵形式的图片拉伸成一列 -1自动计算维度，20*10*10=2000

        x = self.fc1(x) #输入：batch*2000 输出：batch*500
        x = F.relu(x)

        x = self.fc2(x) #输入：batch*500 输出：batch*10

        output = F.log_softmax(x, dim=1) #计算分类后，每个数字的概率值

        return output #返回概率值

#定义optimizer优化器 作用更新模型参数，使得最终的训练和测试的结果达到最优值
model = Digit().to(DEVICE)
optimizer = optim.Adam(model.parameters())

#定义训练的函数方法
def train_model(model, device, train_loader, optimizr, epoch):
    #模型训练
    model.train() #调用模型的train方法开始训练
    #batch_index每次读取下标， target是标签例如图片是5,则标签是5.
    for batch_index, (data, target) in enumerate(train_loader):
        #把训练部署到设备上去
        data, target = data.to(device), target.to(device)
        #梯度初始化为0
        optimizer.zero_grad()
        #训练后的结果
        output = model(data)
        #计算损失 损失用function中的函数cross_entropy计算，cross_entropy是交叉熵损失，交叉熵损失适用于多分类的任务
        loss = F.cross_entropy(output, target)
        #找到概率值最大的下标
        pred = output.max(1, keepdim=True)
        #反向传播
        loss.backward()
        #参数的更新优化
        optimizer.step()
        #每隔3000张图片打印一次结果
        if batch_index % 3000 == 0:
            #format是进行格式化的转换
            print("Train Epoch : {}\t Loss : {:.6f}".format(epoch,loss.item()))
# 定义测试方法
def test_model(model, device, test_loader):
    # 模型验证
    model.eval()
    #正确率初始化
    correct = 0.0
    #测试损失初始化
    test_loss = 0.0
    with torch.no_grad():#不计算梯度，也不会进行反向传播
        for data, target in test_loader:
            #部署到device上
            data, target = data.to(device), target.to(device)
            #测试后的结果
            output = model(data)
            #累计测试损失 损失用function中的函数cross_entropy计算，cross_entropy是交叉熵损失，交叉熵损失适用于多分类的任务
            test_loss += F.cross_entropy(output, target).item()
            #找到概率值最大的下标 
            pred = output.max(1, keepdim=True)[1]
            #累计正确的值
            correct += pred.eq(target.view_as(pred)).sum().item()
        #测试损失率
        test_loss /= len(test_loader.dataset)
        #正确率
        correct  /= len(test_loader.dataset)
        #{:.4f} 表示小数点后4位
        print("Test --Average loss:{:.4f},Accuracy : {:.3f}\n".format(
            test_loss, 100.0 * correct))
            # correct 是测试正确的  len（test_loader.dataset）是数据集。相除得到正确率

#调用方法7/8
for epoch in range(1,EPOCHS +1):
    train_model(model,DEVICE,train_loader,optimizer,epoch)
    test_model(model,DEVICE,test_loader)

Train Epoch : 1	 Loss : 2.308326
Test --Average loss:0.0007,Accuracy : 98.460

Train Epoch : 2	 Loss : 0.013952
Test --Average loss:0.0006,Accuracy : 98.680

Train Epoch : 3	 Loss : 0.031765
Test --Average loss:0.0006,Accuracy : 98.710

Train Epoch : 4	 Loss : 0.007598
Test --Average loss:0.0005,Accuracy : 99.030

Train Epoch : 5	 Loss : 0.001636
Test --Average loss:0.0006,Accuracy : 98.780

Train Epoch : 6	 Loss : 0.059467
Test --Average loss:0.0006,Accuracy : 98.940

Train Epoch : 7	 Loss : 0.000427
Test --Average loss:0.0006,Accuracy : 99.000

Train Epoch : 8	 Loss : 0.003582
Test --Average loss:0.0007,Accuracy : 98.930

Train Epoch : 9	 Loss : 0.001118
Test --Average loss:0.0008,Accuracy : 98.760

Train Epoch : 10	 Loss : 0.000021
Test --Average loss:0.0008,Accuracy : 99.000

Train Epoch : 11	 Loss : 0.000002
Test --Average loss:0.0005,Accuracy : 99.250

Train Epoch : 12	 Loss : 0.000207
Test --Average loss:0.0008,Accuracy : 98.880

Train Epoch : 13	 Loss : 0.000074
Test --Average 

In [8]:
!nvidia-smi

Thu Nov  4 00:42:51 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    58W / 149W |    635MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------